In [3]:
import json
import os
file_path = '/Users/jeremiahetiosaomeike/Desktop/Classwork/NLP (COS484)/final/MolT5/ChEBI-20_data/train.txt'
json_path = '/Users/jeremiahetiosaomeike/Desktop/Classwork/NLP (COS484)/final/MolT5/ChEBI-20_train_sft'
dset = []
def process_data_for_training(f_path: str, json_path: str):
    train_json = '/train.json'
    test_json = '/test.json'
    with open(f_path, 'r') as file:
        for i, line in enumerate(file):
            if i == 0:
                continue # ignore that top thingy 

            this_line = line.strip().split('\t') # everything's tab seperated
            if len(this_line) >= 3: # just in case?
                data = {
                    'smiles': this_line[1],
                    'description': this_line[2]
                }
                dset.append(data)
    
    if not os.path.exists(json_path):
        os.mkdir(json_path)
    
    json_path_save = json_path + train_json
    with open(json_path_save, 'w') as json_file:
        json.dump(dset, json_file, indent=4) # need 4 to make parseable

process_data_for_training(file_path, json_path)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

with open('output_data.json', 'r') as file:
    data = json.load(file)

# sep token works better 
training_data = [f"{entry['smiles']} [SEP] {entry['description']}" for entry in data]

# with open('json_sft_train_as_text.txt', 'w') as file:
#     for item in training_data:
#         file.write(f"{item}\n")

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2') # so we can get logits 
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Tokenizer add special token if you're using one that's not in the model's vocabulary
tokenizer.add_special_tokens({'additional_special_tokens': ['[SEP]']})
model.resize_token_embeddings(len(tokenizer))

# Prepare dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='formatted_for_training.txt',
    block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./saved_gpt2',           # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=2,    # batch size for training
    save_steps=100,                   # after # steps model is saved
    save_total_limit=2,               # Only last 2 models are saved
    prediction_loss_only=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Start training
trainer.train()
